# Deliverable Notebook

In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows', None)

Drug Classes:
- A.C.T.H.
- A05A1 CHOLERETICS+CHOLEKINETIC
- ADHD
- ALL OTHER RESPIRATORY
- ALL OTHER THERAPEUTICS
- ALL OTHER UROLOGICAL
- ALLERGY, SYSTEMIC & NASAL
- ANTI-ALZHEIMERS
- ANTI-ANAEMICS, IRON AND ALL COMBINATIONS
- ANTI-INFLAMMATORY AND DIGESTIVE ENZYMES AND MODULA
- ANTI-OBESITY PREPS, EXCL. DIETETICS
- ANTI-PARASITICS, ANTIMALARIALS, INSECTICIDES
- ANTI-ULCERANTS
- ANTIBACTERIALS
- ANTICOAGULANTS
- ANTIDIABETICS
- ANTIGOUT PREPS
- ANTIHYPERTENSIVES, PLAIN & COMBO
- ANTITUBERCULARS
- ANTIVIRALS, HERPES
- BISPHOSPHONATES TUMOR-RELATED & BONY METASTASES
- BLOOD COAGULATION
- BPH (BENIGN PROSTATIC HYPERTROPHY)
- CANCER DETOX AG, ANTI-NAUSEANTS
- CORTICOSTEROIDS, PLAIN & COMBO
- COUGH COLD, INCL FLU ANTIVIRALS
- DERMATOLOGICS
- DIAGNOSTIC EQUIP & ACCESSORIES & ALLERGEN TESTS
- DIETETICS
- EAR TREATMENTS&EYE-EAR COMBOS
- ERECTILE DYSFUNCTION
- ERYTHROPOIETINS
- GENITO-URINARY ANTIBACTERIALS & ANTISEPTICS & ANTI
- GI PRODUCTS
- GROWTH HORMONES
- HEMATOPOIETIC GROWTH FACTORS
- HIV ANTIVIRALS
- HORMONAL CONTRACEPTION, SYSTEMIC&TOPICAL
- HOSPITAL SOLUTIONS
- HYPOTHALAMIC HORMONES
- IMAGING
- IMMUNOLOGY
- IMMUNOSUPPRESSANTS
- INTERFERONS, EXCL MS MARKET
- J07D6 TUBERCULOSIS VACCINES
- LABOUR INDUCERS
- LIPID REGULATORS
- MENTAL HEALTH
- MIOTICS+ANTIGLAUCOMA PREPS
- MULTIPLE SCLEROSIS
- NASAL PREPS, TOPICAL
- NERVOUS SYSTEM DISORDERS
- OCULAR ANTINEOVASCULARISATION
- ONCOLOGICS
- OPHTHALMOLOGY, GENERAL
- OSTEOPOROSIS
- OTHER ALIMENTARY TRACT AND METABOLISM PRODUCTS
- OTHER CARDIOVASCULARS
- OTHER CNS
- OTHER HAEMATOLOGICALS
- OTHER HORMONES
- OTHER WOMEN'S HEALTH
- PAIN
- POLYVAL IMMUNOGLOBLULINS IV&IM
- RESP ANTIVIRALS EX FLU
- RESPIRATORY AGENTS
- SEX HORMONES (ANDROGENS, OESTROGENS, PROGESTOGENS,
- SPEC IMMUNOGLOBULINS & ANTITOXIC SERA
- SYST ANTIFUNGALS
- THROMBOPOIETIN AGONISTS
- THYROID ANTI-THYROID AND IODINE PREPS
- URINARY INCONTINENCE
- VACCINES (PURE, COMB, OTHER)
- VIRAL HEPATITIS
- VITAMINS & MINERALS

In [2]:
drug_class = 'ONCOLOGICS'

In [3]:
# Retrieving all data about the drug class
data = pd.read_csv('../Processed_Data/drugs_pct_changes_monthly.csv')
data = data[data['Major Class'] == drug_class]
data.Date = pd.to_datetime(data.Date, format='%Y_%m')

# Only NDCs that have full period
last_period = data[(data.Year == 2020) & (data.Month == 7)].NDC.unique()
first_period = data[(data.Year == 2014) & (data.Month == 8)].NDC.unique()
selected_NDC = list(set(last_period).intersection(first_period)) 
data = data[data.NDC.isin(selected_NDC)]

# Removing the LOE period
# Getting rid of NDC that do not have LOE date
data = data[data['Estimated LOE Date'] != 'Unspecified']

# Getting rid of NDC that are too close to their LOE date
data['Estimated LOE Date'] = pd.to_datetime(data['Estimated LOE Date'], format='%b-%y')
data = data[data['Estimated LOE Date'] >= pd.to_datetime('2020-07-01')]


# Retrieving year over year percent change data
yy_data = pd.read_csv('../Year Over Year/df_yearoveryear_class.csv')
yy_data.Date = pd.to_datetime(yy_data.Date, format='%Y_%m')

# Retrieving computed predictions
prophet_1 = pd.read_csv(f'../Predictive Models/Results/Method 1/{drug_class}.csv')
prophet_1 = prophet_1.rename(columns={'Unnamed: 0':'date', 'Number':'# price changes prediction', 'Percentage of drugs change price':'Percent # price changes prediction'})
prophet_1.date = pd.to_datetime(prophet_1.date)

prophet_2 = pd.read_csv(f'../Predictive Models/Results/Method 2/{drug_class}.csv')
prophet_2.date = pd.to_datetime(prophet_2.date)

In [4]:
# Extracting number of unique drug per month
nb_drugs = data.groupby('Date')['NDC'].nunique().to_frame(name='# drugs').reset_index().rename(columns={'Date': 'date'})

# Extracting year over year percent change in Sales weighted WAC
yy_change = yy_data[yy_data['Major Class'] == drug_class][['Date', 'Class_wght_pct_change_y/y']].reset_index(drop=True).rename(columns={'Date': 'date', 'Class_wght_pct_change_y/y': 'Y/Y Percent change'})

# Extracting number of changes per month
nb_price_change = data.groupby('Date')['Changed'].sum().to_frame(name='# price changes').reset_index().rename(columns={'Date': 'date'})

# Helper function
def weigthed_average(data, quantity, weights):
    try:
        return (data[quantity] * data[weights]).sum() / data[weights].sum()
    except ZeroDivisionError:
        return data[quantity]    
    
# Extracting Sales weighted WAC per month
data['Sales'] = data['WAC'] * data['TRx']
sales_wac = data.groupby('Date').apply(weigthed_average, 'WAC', 'Sales').to_frame(name = 'Sales weighted WAC').reset_index().rename(columns={'Date': 'date'})

# Extracting TRx weighted WAC per month
trx_wac = data.groupby('Date').apply(weigthed_average, 'WAC', 'TRx').to_frame(name = 'TRx weighted WAC').reset_index().rename(columns={'Date': 'date'})

# Extracting Prophet 2 predictions
prophet_2 = prophet_2[['date', 'yhat']].rename(columns={'yhat': 'Sales weighted WAC predictions'})

# Merging all and formatting
deliverable = nb_drugs.merge(yy_change, how='outer', on='date')
deliverable = deliverable.merge(nb_price_change, how='outer', on='date')
deliverable = deliverable.merge(sales_wac, how='outer', on='date')
deliverable = deliverable.merge(trx_wac, how='outer', on='date')
deliverable = deliverable.merge(prophet_1, how='outer', on='date')
deliverable = deliverable.merge(prophet_2, how='outer', on='date')

# Formatting date
deliverable['Year'] = deliverable.date.dt.year
deliverable['Month'] = deliverable.date.dt.month
deliverable.drop('date', axis=1, inplace=True)

# Creating handy class column
deliverable['Class'] = drug_class

# Computing percentage nb of drugs that have price change
deliverable['Percent # price changes'] = deliverable['# price changes'] / deliverable['# drugs']

deliverable = deliverable[['Class', 'Year', 'Month', 'Y/Y Percent change', '# drugs','# price changes', '# price changes prediction', 'Percent # price changes', 'Percent # price changes prediction',  'TRx weighted WAC', 'Sales weighted WAC', 'Sales weighted WAC predictions']]
deliverable = deliverable[deliverable.Year != 2014]

In [5]:
deliverable

,Class,Year,Month,Y/Y Percent change,# drugs,# price changes,# price changes prediction,Percent # price changes,Percent # price changes prediction,TRx weighted WAC,Sales weighted WAC,Sales weighted WAC predictions
5,ONCOLOGICS,2015,1,NaN,94.0,36.0,3377.0,0.382979,0.387283,422.420121,774.231051,752.545558
6,ONCOLOGICS,2015,2,NaN,94.0,1.0,193.0,0.010638,0.022099,418.510830,781.920046,774.121647
7,ONCOLOGICS,2015,3,NaN,94.0,18.0,1389.0,0.191489,0.159341,427.711603,777.865908,773.811961
8,ONCOLOGICS,2015,4,NaN,94.0,5.0,715.0,0.053191,0.081967,424.243316,782.906961,788.707130
9,ONCOLOGICS,2015,5,NaN,94.0,4.0,238.0,0.042553,0.027322,423.418137,766.221449,780.101036
10,ONCOLOGICS,2015,6,NaN,94.0,30.0,1585.0,0.319149,0.181818,429.679033,772.181836,783.372875
11,ONCOLOGICS,2015,7,NaN,94.0,17.0,1476.0,0.180851,0.169312,417.648729,748.549134,764.580006
12,ONCOLOGICS,2015,8,0.092108,94.0,0.0,46.0,0.000000,0.005291,421.884136,766.953066,780.489509
13,ONCOLOGICS,2015,9,0.099199,94.0,8.0,643.0,0.085106,0.073684,437.567252,795.652162,798.120325
14,ONCOLOGICS,2015,10,0.101094,94.0,25.0,1461.0,0.265957,0.167513,449.096580,806.968064,803.122122


In [6]:
deliverable.to_csv(f'deliverable_{drug_class}.csv', index=False)